# DMTA Basics: E* ↔ G* Modulus Conversion

This notebook demonstrates how to use RheoJAX with DMTA (Dynamic Mechanical Thermal Analysis) data.

## Learning Objectives

After completing this notebook, you will be able to:
- Understand the relationship between E* and G* moduli
- Use `convert_modulus()` for manual conversion
- Fit E* data directly with `deformation_mode='tension'`
- Query DMTA-compatible models from the registry

## Prerequisites

Basic familiarity with RheoJAX model fitting (see `examples/basic/01-maxwell-fitting.ipynb`).

**Estimated Time:** 15 minutes

In [ ]:
import os
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np

# Add parent directory if needed
if os.path.abspath(os.path.join(os.getcwd(), '../..')) not in sys.path:
    sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../..')))

from rheojax.core.jax_config import safe_import_jax

jax, jnp = safe_import_jax()

np.random.seed(42)
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
warnings.filterwarnings('ignore', category=RuntimeWarning)

# FAST_MODE for CI
FAST_MODE = os.environ.get('FAST_MODE', '1') == '1'
print(f'FAST_MODE: {FAST_MODE}')

## 1. The Physics: E* = 2(1+ν) · G*

DMTA instruments apply tensile, bending, or compression deformations and measure the **Young's modulus** E*(ω). Rotational rheometers apply shear and measure the **shear modulus** G*(ω). For isotropic, linear-viscoelastic materials:

$$E^*(\omega) = 2(1 + \nu) \cdot G^*(\omega)$$

where ν is Poisson's ratio:
- **Rubbers/elastomers** (T >> Tg): ν ≈ 0.50 → E* = 3G*
- **Glassy polymers** (T << Tg): ν ≈ 0.35 → E* ≈ 2.7G*

In [ ]:
# Generate synthetic Maxwell G*(ω) data
omega = np.logspace(-2, 3, 100)
G0 = 1e6    # Shear modulus (Pa)
eta = 1e4   # Viscosity (Pa·s)
tau = eta / G0  # Relaxation time

omega_tau = omega * tau
G_prime = G0 * omega_tau**2 / (1 + omega_tau**2)
G_double_prime = G0 * omega_tau / (1 + omega_tau**2)
G_star = G_prime + 1j * G_double_prime

# Convert to E* (rubber, ν = 0.5)
nu = 0.5
factor = 2 * (1 + nu)  # = 3.0
E_star = factor * G_star
E_prime = factor * G_prime
E_double_prime = factor * G_double_prime

print(f'Conversion factor for ν={nu}: {factor:.1f}')
print(f'G0 = {G0:.0e} Pa → E0 = {G0*factor:.0e} Pa')

In [ ]:
# Compare G* and E* visually
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.loglog(omega, G_prime, 'b-', lw=2, label="G' (shear)")
ax1.loglog(omega, G_double_prime, 'b--', lw=2, label='G" (shear)')
ax1.loglog(omega, E_prime, 'r-', lw=2, label="E' (tensile)")
ax1.loglog(omega, E_double_prime, 'r--', lw=2, label='E" (tensile)')
ax1.set_xlabel('ω (rad/s)')
ax1.set_ylabel('Modulus (Pa)')
ax1.set_title('G* vs E* for Maxwell Model')
ax1.legend()

# Ratio E/G should be constant = 3.0
ratio = np.abs(E_star) / np.abs(G_star)
ax2.semilogx(omega, ratio, 'k-', lw=2)
ax2.axhline(3.0, color='gray', ls='--', label=f'2(1+ν) = {factor:.1f}')
ax2.set_xlabel('ω (rad/s)')
ax2.set_ylabel('|E*| / |G*|')
ax2.set_title('Modulus Ratio (should be constant)')
ax2.set_ylim(2.5, 3.5)
ax2.legend()

plt.tight_layout()
plt.close('all')

## 2. Manual Conversion with `convert_modulus()`

The `rheojax.utils.modulus_conversion` module provides the `convert_modulus()` utility for array-level conversion.

In [ ]:
from rheojax.utils.modulus_conversion import POISSON_PRESETS, convert_modulus

# Show available presets
print('Poisson ratio presets:')
for material, nu_val in POISSON_PRESETS.items():
    factor_val = 2 * (1 + nu_val)
    print(f'  {material:20s}: ν = {nu_val:.2f}  →  E/G = {factor_val:.2f}')

In [ ]:
# E* → G* conversion (rubber)
G_from_E = convert_modulus(E_star, 'tension', 'shear', poisson_ratio=0.5)
np.testing.assert_allclose(np.abs(G_from_E), np.abs(G_star), rtol=1e-10)
print('E→G conversion: EXACT match ✓')

# Roundtrip: E → G → E
E_roundtrip = convert_modulus(G_from_E, 'shear', 'tension', poisson_ratio=0.5)
np.testing.assert_allclose(np.abs(E_roundtrip), np.abs(E_star), rtol=1e-10)
print('Roundtrip E→G→E: EXACT match ✓')

# Same-mode conversion is a no-op
G_same = convert_modulus(G_star, 'shear', 'shear')
np.testing.assert_array_equal(G_same, G_star)
print('Same-mode conversion: no-op ✓')

## 3. Automatic Conversion in `model.fit()`

The preferred approach: pass E* data directly to `model.fit()` with `deformation_mode='tension'`. The model converts E*→G* internally, fits in G-space, and `predict()` converts back to E*.

In [ ]:
from rheojax.models.classical.maxwell import Maxwell

# Fit with E* data directly
model = Maxwell()
model.fit(
    omega, E_star,
    test_mode='oscillation',
    deformation_mode='tension',
    poisson_ratio=0.5,
)

# Parameters are in G-space (model-native)
fitted_G0 = model.parameters.get_value('G0')
fitted_eta = model.parameters.get_value('eta')

print(f'True G0  = {G0:.0e} Pa')
print(f'Fitted G0 = {fitted_G0:.0e} Pa  (error: {abs(fitted_G0-G0)/G0*100:.2f}%)')
print(f'\nTrue eta  = {eta:.0e} Pa·s')
print(f'Fitted eta = {fitted_eta:.0e} Pa·s  (error: {abs(fitted_eta-eta)/eta*100:.2f}%)')

In [ ]:
# predict() returns E* (same space as input)
E_pred = model.predict(omega, test_mode='oscillation')

fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(omega, np.real(E_star), 'ro', ms=4, alpha=0.5, label="E' data")
ax.loglog(omega, np.imag(E_star), 'bs', ms=4, alpha=0.5, label='E" data')
ax.loglog(omega, np.real(E_pred), 'r-', lw=2, label="E' fit")
ax.loglog(omega, np.imag(E_pred), 'b-', lw=2, label='E" fit')
ax.set_xlabel('ω (rad/s)')
ax.set_ylabel('Modulus (Pa)')
ax.set_title('Maxwell Fit to DMTA Data (E*)')
ax.legend()
plt.tight_layout()
plt.close('all')

# Verify predict is in E-space (factor of 3 from G-space)
G_pred = model._predict(omega)
np.testing.assert_allclose(np.abs(E_pred), np.abs(G_pred) * 3.0, rtol=1e-6)
print('predict() returns E-space: ✓')

## 4. Deformation Mode in RheoData

RheoData objects carry deformation mode metadata, enabling auto-detection.

In [ ]:
from rheojax.core.data import RheoData

# Default is shear
data_shear = RheoData(x=omega, y=G_star, validate=False)
print(f'Default mode: {data_shear.deformation_mode}')
print(f'Labels: {data_shear.storage_modulus_label}, {data_shear.loss_modulus_label}')

# Tension mode
data_tension = RheoData(
    x=omega, y=E_star,
    metadata={'deformation_mode': 'tension'},
    validate=False,
)
print(f'\nTension mode: {data_tension.deformation_mode}')
print(f'Labels: {data_tension.storage_modulus_label}, {data_tension.loss_modulus_label}')

## 5. Registry Query: DMTA-Compatible Models

The `ModelRegistry.find()` method supports filtering by deformation mode.

In [ ]:
from rheojax.core.inventory import Protocol
from rheojax.core.registry import ModelRegistry
from rheojax.core.test_modes import DeformationMode

# All models supporting oscillation + tension
tension_models = ModelRegistry.find(
    protocol=Protocol.OSCILLATION,
    deformation_mode=DeformationMode.TENSION,
)
print(f'{len(tension_models)} models support DMTA (oscillation + tension):')
for m in sorted(tension_models):
    print(f'  - {m}')

# Shear-only models
all_models = set(ModelRegistry.list_models())
shear_only = all_models - set(tension_models)
print(f'\n{len(shear_only)} shear-only models (no DMTA):')
for m in sorted(shear_only):
    print(f'  - {m}')

## Key Takeaways

- **E* = 2(1+ν)·G***: The relaxation spectrum is a material property — only the amplitude scale changes
- **`deformation_mode='tension'`**: Pass E* data directly, model works in G-space internally
- **`convert_modulus()`**: For manual array-level conversion
- **41+ models**: All oscillation-capable models work with DMTA data automatically
- **Poisson's ratio**: 0.5 for rubbers, ~0.35 for glassy polymers

## Next Steps

- `02_dmta_master_curve.ipynb`: TTS with multi-temperature DMTA data
- `03_dmta_fractional_models.ipynb`: Fractional models for glass transition fitting